In [18]:
#!pip install pandas openpyxl

In [19]:
import pandas as pd
df50_closingPrice = pd.read_excel(r"上证50股指期货收盘价(主力合约)2022-12-31.xlsx", engine='openpyxl')


In [20]:
df50_closingPrice.head()

,指标名称,上证50股指期货:收盘价(主力合约)
0,频率,日
1,单位,元/张
2,指标ID,M004349669
3,2022-12-30,2646.4
4,2022-12-29,2630.2


In [21]:
#delete first three rows 
df50_closingPrice = df50_closingPrice.iloc[3:]
#change feature name to "date" and "close"
df50_closingPrice.columns = ["date", "close"]
df50_closingPrice.head()

,date,close
3,2022-12-30,2646.4
4,2022-12-29,2630.2
5,2022-12-28,2647.4
6,2022-12-27,2642.6
7,2022-12-26,2614


In [22]:
df50_closingPrice['date']=pd.to_datetime(df50_closingPrice['date'])
df50_closingPrice.set_index("date", inplace=True)
weekly_groups=df50_closingPrice.groupby(pd.Grouper(freq='W'))
weekly_closingPrice=weekly_groups.mean()
weekly_closingPrice


,close
date,
2015-04-19,3268.90
2015-04-26,3237.92
2015-05-03,3302.25
2015-05-10,3160.04
2015-05-17,3123.04
...,...
2022-12-04,2596.28
2022-12-11,2683.24
2022-12-18,2684.16


In [23]:
#add index week1 week2 week3...
weekly_closingPrice['week'] = range(1, len(weekly_closingPrice) + 1)
weekly_closingPrice


,close,week
date,,
2015-04-19,3268.90,1
2015-04-26,3237.92,2
2015-05-03,3302.25,3
2015-05-10,3160.04,4
2015-05-17,3123.04,5
...,...,...
2022-12-04,2596.28,399
2022-12-11,2683.24,400
2022-12-18,2684.16,401


In [24]:
# make close column value natural log of close
import numpy as np
weekly_closingPrice['close'] = np.log(weekly_closingPrice['close'])
weekly_closingPrice

,close,week
date,,
2015-04-19,8.092209,1
2015-04-26,8.082686,2
2015-05-03,8.102359,3
2015-05-10,8.058340,4
2015-05-17,8.046562,5
...,...,...
2022-12-04,7.861835,399
2022-12-11,7.894780,400
2022-12-18,7.895123,401


In [25]:
#check missing value
weekly_closingPrice.isnull().sum()
#find out missing value
weekly_closingPrice[weekly_closingPrice.isnull().values==True]


,close,week
date,,
2016-02-14,NaN,44
2016-10-09,NaN,78
2017-10-08,NaN,130
2018-10-07,NaN,182
2019-02-10,NaN,200
2020-02-02,NaN,251
2022-02-06,NaN,356
2022-10-09,NaN,391


In [26]:
#drop missing value
weekly_closingPrice.dropna(inplace=True)


In [27]:
weekly_closingPrice.set_index("week", inplace=True)
weekly_closingPrice

,close
week,
1,8.092209
2,8.082686
3,8.102359
4,8.058340
5,8.046562
...,...
399,7.861835
400,7.894780
401,7.895123


In [28]:
#diff function
weekly_closingPrice['closingPrice'] = weekly_closingPrice['close'].diff(periods=1)
weekly_closingPrice

,close,closingPrice
week,,
1,8.092209,NaN
2,8.082686,-0.009522
3,8.102359,0.019673
4,8.058340,-0.044019
5,8.046562,-0.011778
...,...,...
399,7.861835,0.024975
400,7.894780,0.032945
401,7.895123,0.000343


In [29]:
#delete closing_price column
del weekly_closingPrice['close']
weekly_closingPrice

,closingPrice
week,
1,NaN
2,-0.009522
3,0.019673
4,-0.044019
5,-0.011778
...,...
399,0.024975
400,0.032945
401,0.000343


In [30]:
#to csv
weekly_closingPrice.to_csv(r'50_Future_return_weekly.csv', index = False, header=True)
